<a href="https://colab.research.google.com/github/junho328/Recommender-System-NLP/blob/main/Recommender_System_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference: 김도형의 '데이터 사이언스 스쿨'

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 4.4 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633965 sha256=3682bd0ab38482273d7dedb106ee832f39e4ced03eecf74b646ed7a6a36fb8ed
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
import surprise
import pandas as pd
import numpy as np

In [ ]:
#MovieLense 데이터 다운로드
data = surprise.Dataset.load_builtin('ml-100k')

In [ ]:
df = pd.DataFrame(data.raw_ratings, columns = ['user','item','rate','id'])
del df['id']
df.head(10)

,user,item,rate
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0
5,298,474,4.0
6,115,265,2.0
7,253,465,5.0
8,305,451,3.0
9,6,86,3.0


In [ ]:
# x축이 상품, y축이 사용자 아이디인 평점 행렬(Rate matrix) 만들기

df_table = df.set_index(['user','item']).unstack()
df_table.shape

(943, 1682)

*unstack(): 인덱스를 컬럼으로 바꾼다

In [ ]:
df_table.iloc[212:222,808:817].fillna("")

rate                                        
item  211  212  213  214  215  216  217  218  219
user                                             
290   3.0                      4.0       2.0     
291        4.0       4.0  4.0            4.0  4.0
292                  3.0                         
293   4.0       3.0       4.0  4.0  3.0  2.0     
294                                              
295             5.0       5.0  5.0  4.0  5.0     
296   4.0                                        
297   4.0       3.0       2.0  4.0       3.0     
298   5.0       3.0       5.0                    
299   4.0  4.0  5.0            5.0

#추천 시스템이란?
- 사용자 아이디와 상품 아이디라는 두 개의 카테고리 입력과 평점 출력을 가지는 예측 시스템

## 추천 시스템 알고리즘
1. Baseline Model
2. Collaborative Filtering(CF)
  - 2-1. Neighborhood Models
    - User-based CF
    - Item-based Cf
  - 2-2. Latent Factor Models
    - Matrix Factorization
    - SVD

3. Content-Based Recommendation

### 1. Baseline Model
- 가장 단순한 회귀분석모형
- 사용자와 상품 특성에 의한 평균 평점의 합으로 예측치 계산

- 사용법
  1. 데이터세트의 split,folds 메소드를 사용하여 K-Fold 트레이닝 데이터셋과 테스트 데이터셋을 만든다.
  2. 모형 알고리즘 객체를 생성한다.
  3. 모형 알고리즘 객체의 train 메서드와 트레이닝 데이터셋으로 모수 추정 후, test 메소드로 테스트 데이터셋에 대한 예측을 실시한다.
  4. accuracy 서브패키지의 성능평가 함수를 사용하여 예측 성능을 계산한다.

In [ ]:
from surprise.model_selection import KFold

bsl_options = {
    'method': 'als',
    'n_epochs': 5,
    'reg_u': 12,
    'reg_i': 5
}
algo = surprise.BaselineOnly(bsl_options)

np.random.seed(0)
acc = np.zeros(3)
cv = KFold(3)
for i, (trainset, testset) in enumerate(cv.split(data)):
  algo.fit(trainset)
  predictions = algo.test(testset)
  acc[i] = surprise.accuracy.rmse(predictions, verbose=True)

acc.mean()

Estimating biases using als...
RMSE: 0.9453
Estimating biases using als...
RMSE: 0.9377
Estimating biases using als...
RMSE: 0.9500


0.9443304984013942

In [ ]:
# 또는 cross_validate 사용
from surprise.model_selection import cross_validate
cross_validate(algo, data)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([0.9384446 , 0.94651657, 0.93612815, 0.94221861, 0.94428787]),
 'test_mae': array([0.74477853, 0.75124267, 0.73975393, 0.745764  , 0.74659098]),
 'fit_time': (0.22449827194213867,
  0.28653383255004883,
  0.2543036937713623,
  0.2371232509613037,
  0.24049949645996094),
 'test_time': (0.13457989692687988,
  0.4352445602416992,
  0.09785008430480957,
  0.09356880187988281,
  0.09232234954833984)}

##Collaborative Filter
- 모든 사용자의 데이터를 균일하게 사용하지 않음
- 평점 행렬이 가진 특정한 패턴을 찾아서 이를 평점 예측에 사용
- Neighborhood Models & Latent Factor Models

###Neighborhood Models
- Memory-based CF
- 특정 사용자의 평점 예측을 위해 사용하는 것이 아님
- 해당 사용자와 유사한 사용자에 대해 가중치를 두는 것
  1. User-based CF: 평점 행렬에서 유사한 사용자 행 벡터를 찾아서 이를 기반으로 빈 데이터를 계산
  2. Item-based CF: 평점 행렬에서 상품 열 벡터의 유사성을 찾고 특정 상품과 유사한 평점 정보를 가지는 상품들로 빈 데이터를 예측 

###### KNN 가중치 예측 방법
- 유사도를 구한 후, 평점을 예측하고자 하는 사용자(또는 상품)와 유사도가 큰 k개의 사용자(또는 상품) 벡터를 상요하여 가중 평균을 구해서 가중치를 예측

In [ ]:
# MSD(Mean Squared Difference) 유사도 계산
sim_options = {'name':'msd'}
algo = surprise.KNNBasic(sim_options=sim_options)
cross_validate(algo, data)['test_mae'].mean()

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


0.7726801901092284

In [ ]:
#Cosine 유사도 계산
sim_options = {'name':'cosine'}
algo = surprise.KNNBasic(sim_options=sim_options)
cross_validate(algo,data)['test_mae'].mean()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


0.8046567723959086

In [ ]:
#Pearson 유사도 계산
sim_options = {'name':'pearson'}
algo = surprise.KNNBasic(sim_options=sim_options)
cross_validate(algo,data)['test_mae'].mean()

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


0.8032778978216127

In [ ]:
#Pearson-Baseline 유사도 계산
sim_options = {'name':'pearson_baseline'}
algo = surprise.KNNBasic(sim_options=sim_options)
cross_validate(algo, data)['test_mae'].mean()

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


0.791677323191221

In [ ]:
#KNNwithMeans
sim_options={'name':'pearson_baseline'}
alog = surprise.KNNWithMeans(sim_options=sim_options)
cross_validate(algo,data)['test_mae'].mean()

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


0.7912940714471514

In [ ]:
#KNNBaseline
sim_options={'name':'pearson_baseline'}
algo = surprise.KNNBaseline(sim_options=sim_options)
cross_validate(algo,data)['test_mae'].mean()

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


0.7211484159690038

###Latent Factor Models
- 긴 사용자 특성이나 상품 특성을 몇 개의 요인 벡터로 간략화할 수 있다는 가정
- PCA(Principle Component Analysis)를 사용하여 긴 특성 벡터를 소수의 차원으로 차원 축소할 수 있듯이 사용자의 특성도 차원 축소할 수 있다.

  1.Matrix Factorization : 모든 사용자와 상품에 대해 오차 함수를 최소화하는 요인 벡터를 찾아낸다.

  2.SVD : Matrix Factorization을 푸는 방법 중 하나

In [ ]:
algo = surprise.SVD(n_factors=100)
cross_validate(algo,data)['test_mae'].mean()

0.7365684847263315

In [ ]:
# NMF (Non-negative Matrix Factorization)
algo = surprise.NMF(n_factors=100)
cross_validate(algo, data)['test_mae'].mean()

0.838535743496668